## 1- import and reading data


In [1]:
import sys

sys.path.append('../../../scripts/utilities')
from helper_functions import *
sys.path.append('../../../scripts/data_preprocessing')
from data_cleaning import *

In [2]:
base_path = '../../../data/raw_data/'
raw_dfs_merged = read_files('raw_dfs_merged.csv', base_path=base_path)[0]

## 2- handle missing values

**2.1**  Deleting the entire column with many missing value
**2.2**  Imputing the Missing Value


<span style="color:orange">2.1- Deleting the entire column </span>
>when:
 >>1 - missing value in type Missing At Random (MAR) or Missing Completely At Random (MCAR)
 >>2- column has many missing value

In [3]:
threshold_percentage = 50
raw_dfs_merged_columns_to_drop = extract_columns_by_threshold(raw_dfs_merged, threshold_percentage)
raw_dfs_merged = raw_dfs_merged.drop(columns=raw_dfs_merged_columns_to_drop)
raw_dfs_merged.shape

(10175, 377)

<span style="color:orange">2.2- Imputing the Missing Value </span>
>when:
 >>1 - missing value in type Missing Not At Random (MNAR) in this case (In the cells that used compounds 7 and 9, they express this type)
 >>2- Cells that were not removed by applying threshold in the previous step

__2.2.1 imputing by median(numerical columns) and mod(categorical columns)__

In [4]:
numerical_columns = extract_numerical_columns(raw_dfs_merged)
numerical_columns.remove('SEQN')
numerical_columns[:10]

['SDDSRVYR',
 'RIDSTATR',
 'RIAGENDR',
 'RIDAGEYR',
 'RIDRETH1',
 'RIDRETH3',
 'RIDEXMON',
 'DMQMILIZ',
 'DMDBORN4',
 'DMDCITZN']

In [5]:
categorical_columns = extract_categorical_columns(raw_dfs_merged)
categorical_columns[:10]

['OHX02CTC',
 'OHX03CTC',
 'OHX04CTC',
 'OHX05CTC',
 'OHX06CTC',
 'OHX07CTC',
 'OHX08CTC',
 'OHX09CTC',
 'OHX10CTC',
 'OHX11CTC']

-- imputing numerical columns

In [14]:
df_imp1 = replace_repeating_sequence(raw_dfs_merged, numerical_columns, strategy='median', sequence_values=[7, 9])
df_imp2 = statistical_imputer(df_imp1, numerical_columns)

,SEQN,SDDSRVYR,RIDSTATR,RIAGENDR,RIDAGEYR,RIDRETH1,RIDRETH3,RIDEXMON,DMQMILIZ,DMDBORN4,...,LBDBSELC,LBXBMN,LBDBMNSI,LBDBMNLC,URXVOL1,URDFLOW1,LBDB12,LBDB12SI,MCQ160L,MCQ220
0,73557,8.0,2.0,1.0,69.0,4.0,4.0,1.0,1.0,1.0,...,0.0,9.89,180.00,0.0,87.0,0.821,524.0,386.7,2.0,2.0
1,73558,8.0,2.0,1.0,54.0,3.0,3.0,1.0,2.0,1.0,...,0.0,8.15,148.33,0.0,90.0,1.636,507.0,374.2,2.0,2.0


-- imputing categorical columns

In [15]:
df_imp3 = statistical_imputer(df_imp2, categorical_columns, strategy='most_frequent')
df_imp3.head(2)

,SEQN,SDDSRVYR,RIDSTATR,RIAGENDR,RIDAGEYR,RIDRETH1,RIDRETH3,RIDEXMON,DMQMILIZ,DMDBORN4,...,LBDBSELC,LBXBMN,LBDBMNSI,LBDBMNLC,URXVOL1,URDFLOW1,LBDB12,LBDB12SI,MCQ160L,MCQ220
0,73557,8.0,2.0,1.0,69.0,4.0,4.0,1.0,1.0,1.0,...,0.0,9.89,180.00,0.0,87.0,0.821,524.0,386.7,2.0,2.0
1,73558,8.0,2.0,1.0,54.0,3.0,3.0,1.0,2.0,1.0,...,0.0,8.15,148.33,0.0,90.0,1.636,507.0,374.2,2.0,2.0


## 3- save after dealing with missing values

In [21]:
save_files([df_imp3], 'df_filling_missing_values.csv', base_path='../../../data/processed_data/')